# DATASCI W261: Machine Learning at Scale

## HW13

- **Juanjo Carin**
- [juanjose.carin@ischool.berkeley.edu](mailto:juanjose.carin@ischol.berkeley.com)
- W261-2
- Week 13
- Submission date: 12/09/2015

## Spark initialization

In [1]:
import os
import sys
spark_home = os.environ['SPARK_HOME'] = \
   '/HD/spark/'

if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.2
      /_/

Using Python version 2.7.10 (default, Oct 19 2015 18:04:42)
SparkContext available as sc, HiveContext available as sqlContext.


# HW13.1: Spark implementation of basic PageRank

**Write a basic Spark implementation of the iterative PageRank algorithm that takes sparse adjacency lists as input. Make sure that your implementation utilizes teleportation (1-damping/the number of nodes in the network), and further, distributes the mass of dangling nodes with each iteration so that the output of each iteration is correctly normalized (sums to 1).**

>**[NOTE: The PageRank algorithm assumes that a random surfer (walker), starting from a random web page, chooses the next page to which it will move by clicking at random, with probability d, one of the hyperlinks in the current page. This probability is represented by a so-called ‘damping factor’ d, where d ∈ (0, 1). Otherwise, with probability (1 − d), the surfer jumps to any web page in the network. If a page is a dangling end, meaning it has no outgoing hyperlinks, the random surfer selects an arbitrary web page from a uniform distribution and “teleports” to that page]**

**In your Spark solution, please use broadcast variables and caching to make sure your code is as efficient as possible.**

**As you build your code, use the following test data to check you implementation:**

**s3://ucb-mids-mls-networks/PageRank-test.txt**

**Set the teleportation parameter  to 0.15 (1-d, where d, the damping factor is set to 0.85), and crosscheck your work with the true result, displayed in the first image in the Wikipedia article:**

**https://en.wikipedia.org/wiki/PageRank**

**and here for reference are the corresponding resulting PageRank probabilities:**

    A,0.033
    B,0.384
    C,0.343
    D,0.039
    E,0.081
    F,0.039
    G,0.016
    H,0.016
    I,0.016
    J,0.016
    K,0.016

**Run this experiment locally first. Report the local configuration that you used and how long in minutes and seconds it takes to complete your job.**

In [3]:
import ast
import time
from operator import add

start = time.time()

lines = sc.textFile('PageRank-test.txt')

def adj_lists(line):
    line = line.strip().split('\t')
    source = line[0]
    adj_list = ast.literal_eval(line[1]).keys()
    yield str(source), adj_list
adj_list = lines.\
    flatMap(lambda l: adj_lists(l)).\
    cache()

#from collections import namedtuple
#Adj_List = namedtuple('link', 'source sinks rank')
#def adj_lists(line):
#    line = line.strip().split('\t')
#    source = line[0]
#    adj_list = ast.literal_eval(line[1]).keys()
#    return Adj_List(str(source), adj_list, 0.1)
#sources = lines.map(adj_lists)

sinks = adj_list.\
    values().\
    flatMap(lambda line: line).\
    distinct().\
    map(lambda line: (line, [])).\
    cache()

dangling = sinks.\
    subtractByKey(adj_list).\
    cache()

total_adj_list = adj_list.\
    union(dangling).\
    cache()
num_nodes = total_adj_list.count()

def computeContribs(sinks, score):
    """Calculates URL contributions to the rank of other URLs."""
    num_sinks = len(sinks)
    for s in sinks:
        yield (s, score / num_sinks)
        
scores = total_adj_list.map(lambda (source, sinks): (source, 1.0/num_nodes))

stop = time.time()
exec_time = round(stop-start)
print 'Execution time {}:{}'.format(str(int(exec_time/60)), 
                                    str(int(exec_time) - 
                                        60*int(exec_time/60)).zfill(2))

print dangling.collect()

Execution time 0:31
[('A', [])]


In [ ]:
start = time.time()

for iteration in range(10):
    # Calculates URL contributions to the rank of other URLs.
    mass_dangling = dangling.\
        join(scores).\
        mapValues(lambda (sinks, score): score).\
        values().sum()

    contribs = total_adj_list.\
        join(scores).\
        flatMap(lambda (source, (sinks, score)): computeContribs(sinks, score))
    # Re-calculates URL ranks based on neighbor contributions.
    new_scores = contribs.\
        reduceByKey(add).\
        mapValues(lambda score: (score + mass_dangling/num_nodes) * 0.85 
                  + 0.15/num_nodes)
    poor_scores = scores.\
        subtractByKey(new_scores).\
        mapValues(lambda score: mass_dangling/num_nodes * 0.85 
                  + 0.15/num_nodes)
    scores = new_scores.union(poor_scores)
    print 'ITERATION {}'.format(str(iteration))
    for (node, score) in scores.\
        map(lambda (node, score): (score, node)).\
        sortByKey(ascending = False).\
        map(lambda (score, node): (node, score)).\
        collect():
        print '{}:\t{}'.format(node, score)
    print '--------------'

stop = time.time()
exec_time = round(stop-start)
print 'Execution time {}:{}'.format(str(int(exec_time/60)), 
                                    str(int(exec_time) - 60*int(a/60)).zfill(2))

ITERATION 0
E:	0.329752066116
B:	0.316873278237
C:	0.0979338842975
A:	0.0592975206612
D:	0.0464187327824
F:	0.0464187327824
H:	0.0206611570248
J:	0.0206611570248
G:	0.0206611570248
I:	0.0206611570248
K:	0.0206611570248
--------------
ITERATION 1
C:	0.28756073128
B:	0.260690896569
D:	0.111648196844
F:	0.111648196844
E:	0.0994133483596
A:	0.0379464062109
H:	0.0182184447784
J:	0.0182184447784
G:	0.0182184447784
I:	0.0182184447784
K:	0.0182184447784
--------------
ITERATION 2

**Repeat this experiment on AWS. Report the AWS cluster configuration that you used and how long in minutes and seconds it takes to complete your job. (in your notebook, cat the cluster config file)**

# HW 13.2: Applying PageRank to the Wikipedia hyperlinks network

**Run your Spark PageRank implementation on the Wikipedia dataset for 10 iterations, and display the top 100 ranked nodes (with alpha = 0.85).**

NOTE: Wikipedia data is located on S3 at  s3://ucb-mids-mls-networks/wikipedia/

+ s3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt # Graph
+ s3://ucb-mids-mls-networks/wikipedia/indices.txt               # Page titles and page Ids

**Run your PageRank implementation on the Wikipedia dataset for 50 iterations, and display the top 100 ranked nodes (with teleportation factor of 0.15).**


**Plot the pagerank values for the top 100 pages resulting from the 50 iterations run. Then plot the pagerank values for the same 100 pages that resulted from the 10 iterations run.  Comment on your findings.  Have the top 100 ranked pages changed? Have the pagerank values changed? Explain.**

**Report the AWS cluster configuration that you used and how long in minutes and seconds it takes to complete your job.**

In [ ]:
import ast
from operator import add

lines = sc.textFile('PageRank-test.txt')

def adj_lists(line):
    line = line.strip().split('\t')
    source = line[0]
    adj_list = ast.literal_eval(line[1]).keys()
    yield str(source), adj_list
adj_list = lines.\
    flatMap(lambda l: adj_lists(l)).cache()

#from collections import namedtuple
#Adj_List = namedtuple('link', 'source sinks rank')
#def adj_lists(line):
#    line = line.strip().split('\t')
#    source = line[0]
#    adj_list = ast.literal_eval(line[1]).keys()
#    return Adj_List(str(source), adj_list, 0.1)
#sources = lines.map(adj_lists)

sinks = adj_list.\
    values().\
    flatMap(lambda line: line).\
    distinct().\
    map(lambda line: (line, []))

dangling_nodes = sinks.\
    subtractByKey(adj_list).\
    cache()

total_adj_list = adj_list.\
    union(dangling_nodes).\
    cache()
num_nodes = total.count()

def computeContribs(sinks, score):
    """Calculates URL contributions to the rank of other URLs."""
    num_sinks = len(sinks)
    for s in sinks:
        yield (s, score / num_sinks)
        
scores = total_adj_list.map(lambda (source, sinks): (source, 1.0/num_nodes))

total = total_adj_list.\
    map(lambda (source, adj_list): (source, 
                                    [adj_list, 1.0/num_nodes]))
dangling = dangling_nodes.\
    map(lambda (source, adj_list): (source, 
                                    [adj_list, 1.0/num_nodes]))

for iteration in range(10):
    # Calculates URL contributions to the rank of other URLs.
    mass_dangling = dangling.\
        mapValues(lambda (sinks, score): score).\
        values().sum()
    contribs = total.\
        flatMapValues(lambda (sinks, score): computeContribs(sinks, score))
    # Re-calculates URL ranks based on neighbor contributions.
    new_scores = contribs.\
        reduceByKey(add).\
        mapValues(lambda score: (score + mass_dangling/num_nodes) * 0.85 
                  + 0.15/num_nodes)
    poor_scores = scores.\
        subtractByKey(new_scores).\
        mapValues(lambda score: mass_dangling/num_nodes * 0.85 
                  + 0.15/num_nodes)
    scores = new_scores.union(poor_scores)
    print 'ITERATION {}'.format(str(iteration))
    for (node, score) in scores.\
        map(lambda (node, score): (score, node)).\
        sortByKey(ascending = False).\
        map(lambda (score, node): (node, score)).\
        collect():
        print '{}:\t{}'.format(node, score)
    print '--------------'

In [ ]:
import re, sys
from operator import add


def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls: yield (url, rank / num_urls)


def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]



lines = sc.textFile('prueba.txt')
print lines.collect()

# Loads all URLs from input file and initialize their neighbors.
links = lines.map(lambda urls: parseNeighbors(urls)).groupByKey().cache()

    # Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.
scores = links.map(lambda (source, sinks): (source, 0.1))
print scores.collect()
    # Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in xrange(int(5)):
    # Calculates URL contributions to the rank of other URLs.
    
    
    #links.subtractByKey(ranks).keys().collect()
    
    
    contribs = links.subtractByKey(scores).flatMap(lambda (url, (urls, rank)):
        computeContribs(urls, rank))
    total_mass = ranks.values().sum()
    print total_mass
    total_mass = 0
     # Re-calculates URL ranks based on neighbor contributions.
    scores = contribs.reduceByKey(add).mapValues(lambda score: (score + total_mass/11) * 0.85 + 0.15/11)
# Collects all URL ranks and dump them to console.
for (node, score) in scores.collect():
    print "%s has rank: %s." % (node, score)

In [ ]:
print links.cogroup(scores).map(lambda (u, (u2,s)): s)#.collect()

print links.join(ranks).keys().collect()
print links.join(ranks).flatMap(lambda (url, (urls, rank)): urls).collect()

In [ ]:
print links.subtractByKey(ranks).keys().collect() # SOURCES, NOT SINKS
print ranks.subtractByKey(links).keys().collect() # SINKS, NOT SOURCES = DANGLING
dangling = ranks.subtractByKey(links)
print dangling.values().sum()
#result = pairs.filter(lambda keyValue: len(keyValue[1]) < 20)
print links.keys().collect()
print ranks.keys().collect()
print '+++++'
print links.leftOuterJoin(ranks).keys().collect()
print links.rightOuterJoin(ranks).keys().collect()
print '+++++'
print ranks.values().sum()
print ranks.count()
print links.count()
print '+++++'
print lines.map(lambda urls: parseNeighbors(urls)).count()
print lines.map(lambda urls: parseNeighbors(urls)).distinct().count()

print lines.map(lambda urls: parseNeighbors(urls)).collect()
print lines.map(lambda urls: parseNeighbors(urls)).distinct().collect()

We use 2 functions:

1. One to get the cluster centroid closer to any point in the dataset. It calls the `value` on the broadcast variable `centroidsBroadcast`, as its argument is each of the datapoints.

2. An optional function to plot the current centroids. Its argument will be the broadcast centroids, so there's no need to explicitly mention them.

In [ ]:
import numpy as np

#Calculate which class each data point belongs to
def nearest_centroid(line):
    x = np.array([float(f) for f in line.split(',')])
    #### USE BROADCAST VALUES
    closest_centroid_idx = np.sum((x - centroidsBroadcast.value)**2, axis=1).\
        argmin()
    return (closest_centroid_idx,(x,1))

#plot centroids and data points for each iteration
def plot_iteration(means):
    plt.plot(samples1[:, 0], samples1[:, 1], '.', color = 'blue')
    plt.plot(samples2[:, 0], samples2[:, 1], '.', color = 'blue')
    plt.plot(samples3[:, 0], samples3[:, 1],'.', color = 'blue')
    plt.plot(means[0][0], means[0][1],'*',markersize =10,color = 'red')
    plt.plot(means[1][0], means[1][1],'*',markersize =10,color = 'red')
    plt.plot(means[2][0], means[2][1],'*',markersize =10,color = 'red')
    plt.show()

The driver makes use of both functions above. It first creates K random centroids...that are broadcasted to all executors (probably just 1 in this toy example), and then updates (**broadcasts**) this broadcast variable in each iteration, as the datapoints closer to each centroid change.

In [ ]:
K = 3
# Initialization: initialization of parameter is fixed to show an example
centroids = np.array([[0.0,0.0],[2.0,2.0],[0.0,7.0]])
#### BROADCAST INITIAL CENTROIDS
centroidsBroadcast = sc.broadcast(centroids)

D = sc.textFile("./data.csv").cache()
iter_num = 0
for i in range(10):  
    res = D.map(nearest_centroid).\
        reduceByKey(lambda x,y : (x[0]+y[0],x[1]+y[1])).collect()
    res = sorted(res,key = lambda x : x[0])  #sort based on clusted ID
    centroids_new = np.array([x[1][0]/x[1][1] for x in res])
        #divide by cluster size
    #### USE BROADCAST VALUE TO CHECK CONDITION
    if np.sum(np.absolute(centroids_new-centroidsBroadcast.value))<0.01:
        break
    print "Iteration" + str(iter_num)
    iter_num = iter_num + 1 
    #### UPDATE CENTROIDS AND BROADCAST
    centroidsBroadcast = sc.broadcast(centroids_new)
    #### PRINT NEW CENTROIDS
    print centroidsBroadcast.value
    #### PLOT NEW CENTROIDS
    plot_iteration(centroidsBroadcast.value)
print "Final Results:"
print centroidsBroadcast.value

# HW11.1: Loss Functions

## HW11.1.1
**In the context of binary classification problems, does the linear SVM learning algorithm yield the same result as a L2 penalized logistic regresssion learning algorithm?**

**In your reponse, please discuss the loss functions, and the learnt models, and separating surfaces between the two classes.**

Not necessarily, though if we let both algorithms converge the results (the hyperplanes, and hence the predictions) should be quite similar.

The separatign surface in both algorithms is an hyperplane (specified by an equation of the type $w^T x$, though in the case of the Logistic Regression it represents the logit of the likelihood, i.e., hte logarithm of the Odds Ratio), but the SVM does not intend to minimize any loss function (as the Logistic Regression does), but only the regularization term (similar to the L2 regularization term in the Logistic Regression; in the case of a SVM, the regularization term would be inversely proportional to the margin.

To demonstrate this, **see the addendum at the end of HW11.3.3**.

## HW11.1.2
**In the context of binary classification problems, does the linear SVM learning algorithm yield the same result as a perceptron learning algorithm?**

Same answer as before: the final result (the weights, and hence the decision boundary or hyperplane that makes datapoints to fall into one category or the other) will be similar but not necessarily equal.

# HW11.2: Gradient descent

## HW11.2.1
**In the context of logistic regression describe and define three flavors of penalized loss functions. Are these all supported in Spark MLLib (include online references to support your answers)?**

In Logistic Regression the objective function (or penalized loss function) we want to minimize is defined as:

$$J(w)=\sum_iL(\mathbf{w},\mathbf{x_i},y_i)+\lambda R(\mathbf{w}) = \sum_i log(1+e^{-y \mathbf{w}^T \mathbf{x}_i})+\lambda R(\mathbf{w})$$

where $y \in \{-1,+1\}$ (the definition of the loss term $L(\mathbf{w},\mathbf{x_i},y_i)$ is different when $y \in \{0,+1\}$).

Three flavors of the regularization term would be:
1. **Lasso**, where the L1 norm lf $w$ is used (as $R(\mathbf{w})$: $\left \|  \mathbf{w}\right \|_1$
    - It can help promote sparsity in weights leading to smaller and more interpretable models, the latter of which can be useful for feature selection. 
2. **Ridge**, where the L2 norm of $w$ is used: $\frac{1}{2}\left \|  \mathbf{w}\right \|_2^2$
    - Generally easier to solve than L1-regularized due to smoothness.
3. **Elastic net**, a (weighted) combination of the previous two ones: $\alpha \left \|  w\right \|_1 + (1-\alpha)\frac{1}{2}\left \|  w\right \|_2^2$

All of them are supported in Spark MLLib: http://spark.apache.org/docs/latest/mllib-linear-methods.html#loss-functions

## HW11.2.2
**Descibe probabilitic interpretations of the L1 and L2 priors for penalized logistic regression.**

To make it simple, I will use the case in which there are only 2 features or independent variables.

In Logistic Regression we want to maximize the likelihood of a class given the independent variables: $\Pr(y=+1 \mid \mathbf{x})$. The loss term is the inverse of (the logit of) that prior, and hence we want to minimize it. That loss term is a function of $w_1$ and $w_2$ and thus defined in $\mathbb{R}^2$.

The L1 regularization term adds a constraint in finding the minimum of $L(\mathbf{w},\mathbf{x},y)$: $|w_1| + |w_2|$ cannot exceed a certain value (i.e., $(w_1, w_2)$ is inside a "diamond". That's like adding a prior to the weights: instead of minimizing the cost function (i.e., maximizing the likelihood) given the training data, we are maximizing the likelihood given the *additional information bias*.

The same applies to the L2 regularization term, but in this case the constraint puts $(w_1,w_2)$ inside a circle ($|w_1|^2+|w_2|^2$ cannot exceed a certain value).

-----

L1 and L2 are also called Laplacian and Gaussian priors, respectively. The Laplacian pdf is similar to the Gaussian but it uses the L1 norm (or Manhattan distance) rather than the L2 norm (or Euclidean distance).

# HW11.3: Logistic Regression

## HW11.3.1
**Generate 2 sets of linearly separable data with 100 data points each using the data generation code provided below and plot each in separate plots. Call one the training set and the other the testing set.**

Each data set will have 100 points, 50 in each category.

In [ ]:
def generateData(n):
    """ 
    generates a 2D linearly separable dataset with n samples. 
    The third element of the sample is the label
    """
    from numpy.random import rand ## Seems like this line was missing    
    xb = (rand(n)*2-1)/2-0.5
    yb = (rand(n)*2-1)/2+0.5
    xr = (rand(n)*2-1)/2+0.5
    yr = (rand(n)*2-1)/2-0.5
    inputs = []
    for i in range(len(xb)):
        inputs.append([xb[i],yb[i],1])
        inputs.append([xr[i],yr[i],-1])
    return inputs

In [ ]:
for i in range(2):
    data = generateData(50)
    data = np.array([np.array(xi) for xi in data])
    plt.plot(data[data[:,2]==1, 0], data[data[:,2]==1, 1], 
               'o', color = 'blue')
    plt.plot(data[data[:,2]==-1, 0], data[data[:,2]==-1, 1], 
               'o', color = 'red')
    if i == 0:
        training_set = data
        plt.title('TRAINING SET')
    else:
        testing_set = data
        plt.title('TESTING SET')
    plt.plot([-1,1], [-1,1], linewidth=2.0, color = 'black')
    plt.grid()
    plt.show()

## HW11.3.2
**Modify this data generation code to generating non-linearly separable training and testing datasets (with approximately 10% of the data falling on the wrong side of the separating hyperplane. Plot the resulting datasets.**

Of the 100 points in each dataset, approximately (well, exactly in this case) 10% (5 in the $x>0, y<0$ region and 5 in the $x<0, y>0$ region) will be assigned the wrong label.

Those points can be anywhere in the wrong region (I was not sure if they should be more likely near the origin (and hence the separating hyperplane).

In [ ]:
def generateData2(n):
    """ 
    generates a 2D NON-linearly separable dataset with n samples. 
    The third element of the sample is the label
    """
    from numpy.random import rand ## Seems like this line was missing    
    xb = (rand(n)*2-1)/2-0.5
    yb = (rand(n)*2-1)/2+0.5
    xr = (rand(n)*2-1)/2+0.5
    yr = (rand(n)*2-1)/2-0.5
    inputs = []
    wrong = 0.1
    for i in range(int(round(len(xb)*wrong))):
        ## FIRST 10% OF THE GENERATED POINTS ARE ASSIGNED THE WRONG LABEL
        inputs.append([xb[i],yb[i],-1])
        inputs.append([xr[i],yr[i],1])
    for i in range(int(round(len(xb)*wrong)), len(xb)):
        inputs.append([xb[i],yb[i],1])
        inputs.append([xr[i],yr[i],-1])
    #inputs = np.array([np.array(xi) for xi in inputs])
    #np.random.shuffle(inputs)
    return inputs

In [ ]:
import csv

for i in range(2):
    data = generateData2(50)
    data = np.array([np.array(xi) for xi in data])
    plt.plot(data[data[:,2]==1, 0], data[data[:,2]==1, 1], 
               'o', color = 'blue')
    plt.plot(data[data[:,2]==-1, 0], data[data[:,2]==-1, 1], 
               'o', color = 'red')
    if i == 0:
        training_set = data
        with open('training_set.csv', 'wb') as f:
            writer = csv.writer(f)
            for row in data:
                writer.writerow(row)
        plt.title('TRAINING SET')
    else:
        testing_set = data
        with open('testing_set.csv', 'wb') as f:
            writer = csv.writer(f)
            for row in data:
                writer.writerow(row)
        plt.title('TESTING SET')
    plt.plot([-1,1], [-1,1], linewidth=2.0, color = 'black')
    plt.grid()
    plt.show()

**NOTE: For the remainder of this problem please use the non-linearly separable training and testing datasets.**

## HW11.3.3
**Using MLLib  train up a LASSO logistic regression model with the training dataset and evaluate with the testing set. What a good number of iterations for training the logistic regression model? Justify with plots and words.**

SOME COMMENTS ABOUT THE CONVERGENCE CRITERIA (that I've chosen):

Let's assume that at a certain iteration, we have the following hyperplane equation: $\pi_1: w_1 x_1 + w_2 x_2 + w_3 = 0$.

For another hyperplane, $\pi_2: w_3 x_1 + w_4 x_2 + w_5 = 0$ (say the one in the following iteration), to be the same, the following condition must be met:

$$\exists \gamma  / \gamma \pi_2 = \pi_1$$

I.e.,

$$\left\{\begin{matrix}
\gamma = \frac{w_1}{w_4}\\ 
\gamma = \frac{w_2}{w_5}\\ 
\gamma = \frac{w_4}{w_6}
\end{matrix}\right.$$

I.e, we don't care if a weight doubles from one iteration to the next, as long as the same happens with all the weights.

To relax the condition, I'll consider that 2 hyperplanes are (almost) the same if the 3 relations above are very similar, i.e.

$$\frac{w_1}{w_4} \simeq \frac{w_2}{w_5} \simeq \frac{w_3}{w_6}$$

Let's say that 2 numbers are similar if their ratio is lower than 1.01 or higher than 0.99 (they have increased or decreased 1% at max), i.e.,

$$\frac{\frac{w_i}{w_{i+3}}}{\frac{w_j}{w_{j+3}}} \in [0.99,1.01]$$

for all possible combinations of $i,j \in \{1,N\}$, where $N$ is the number of features plus 1.

This criterion is a bit arbitrary, and it does not ensure that we have achieved the lowest misclassification error possible. But it can be made more strict just by changing that 1% by say 0.1% (that would be the way to go, rather than imposing any criterion about the misclassification error, which we would not know in a real case).

In [ ]:
#### EXAMPLE
prev_w = np.arange(1,4)
print 'Previous w:\t\t\t', prev_w
w = prev_w * (2 + np.random.normal(size=3)/100)
print 'New w:\t\t\t\t', w
ratio = w/prev_w
print 'Gammas:\t\t\t\t', ratio
## We have to compare all possible ratios: 1 with 2, 1 with 3, 2 with 3
import itertools
comb = [x for x in itertools.combinations(range(len(w)),2)]
ratio_of_ratio = np.array([(w[i]/prev_w[i])/(w[j]/prev_w[j]) for (i,j) in comb])
print 'Ratios of gammas:\t\t', ratio_of_ratio
condition = (0.99<=ratio_of_ratio) & (ratio_of_ratio<=1.01)
print 'Check condition of ratios:\t', condition
print 'Check overall condition:\t', condition.all(axis=0)

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, \
    LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
import itertools

iterations = 20
## Define equally spaced colors for each iteration
colors = plt.cm.Set1(np.linspace(0, 1, iterations+1))

def parsePoint(line):
    ## Change labels to 0 and 1 (rather than -1 and 1)
        ## Those are the values accepted by MLLib
    values = [float(x) for x in line.split(',')]
    if values[2] == -1:
        y = 0
    elif values[2] == 1:
        y = 1
    return LabeledPoint(y, values[:2])

## Load and cache both sets
training = sc.textFile("training_set.csv")
testing = sc.textFile("testing_set.csv")
training = training.map(parsePoint).cache()
testing = testing.map(parsePoint).cache()

## Draw the true hyperplane
x1 = [-1, 1]
w = [-1, 1, 0]
x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
plt.figure(figsize=(10,10))
plt.plot(x1, x2, 'b', label="True line", linewidth=2.0, color = 'black', 
         linestyle='--')

prev_w = [float('inf')]*3
#prev_Err = float('inf')

## Try a few iterations (no need for more in this case)
for it in range(iterations+1):
    print 'Iteration: {}'.format(it)
    # Build the model (Lasso/L1, lambda = 0.5)
    model = LogisticRegressionWithLBFGS.train(training, regType="l1", 
                                              regParam=0.01, 
                                              intercept = True, 
                                              iterations = it)
    print model

    # Evaluating the model on testing data
    labelsAndPreds = testing.\
        map(lambda p: (p.label, model.predict(p.features)))
    Err = labelsAndPreds.\
        filter(lambda (v, p): v != p).count() / float(testing.count())
    w = [model.weights[0], model.weights[1], model.intercept]
    x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
    ## Plot the hyperplanes
    plt.plot(x1, x2, color=colors[it], label="After {} iterations".
             format(str(it)), linewidth=1.5)
    
    ## Stop when it converges
    ## Convergence is decided based on the relative change of the weights
        ## An absolute change makes no sense since we don't know a priori
            ## how large the weights will be
        ## The criteria I've chosed is than, on average, the weight vector
            ## has converged when each weight, on average is less than 33%
            ## above/below its previous value
    #if sum([abs(float(a)/b) if abs(float(a)/b) > 1 else abs(float(b)/a) 
    #        for (a,b) in zip(w,prev_w)]) < len(w)+1 \
    #    and trainErr == prev_trainErr:
        ## NEW CRITERIA: None the weights (incl. the intercept) is 25%
            ## above/below its previous value
    #if max([abs(float(a)/b) if abs(float(a)/b) > 1 else abs(float(b)/a) 
    #        for (a,b) in zip(w,prev_w)]) < 1.25 and trainErr == prev_trainErr:
    #    break
    
    comb = [c for c in itertools.combinations(range(len(w)),2)]
    ratio_of_ratio = np.array([(w[i]/prev_w[i])/(w[j]/prev_w[j]) 
                               for (i,j) in comb])
    condition = (0.99<=ratio_of_ratio) & (ratio_of_ratio<=1.01)
    if condition.all(axis=0):
        break
    
    prev_w = w
    #prev_Err = Err
    print "Testing Error: {}".format(str(Err))
    print '------------------'

## Last iteration's results
print "Testing Error: {}".format(str(Err))
print '------------------'
print 'CONVERGENCE AFTER {} ITERATIONS'.format(it)
print 'Relative change of weights (incl. intercept) between iterations '\
    '{} and {}:\n\t{}'.format(it-1, it, [a/b for (a,b) in zip(w,prev_w)])

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, fontsize=20, borderaxespad=0.)
plt.xlabel("x1")
plt.ylabel("x2")
axes = plt.gca()
axes.set_ylim([-1,1])
axes.set_xlim([-1,1])
plt.grid()
plt.show()

The weight vector is relatively close to its final value just after the 2nd iteration. And at that point we've already achieved the lowest testing error possible (10%). The convergence criteria is met after 10 iterations.

Do note that the values of the weights (and the results commented above) are highly dependent on the value of `regParam` ($\lambda$; I used $\lambda = 0.01$).

### Additional
Comparison between Logistic Regression and SVM (using Ridge instead of Lasso).

In [ ]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel

for i in range(10):
    print 'Iteration: {}'.format(i)
    # Build the model
    model_LR = LogisticRegressionWithLBFGS.train(training, regType="l2", 
                                                 regParam=0.01, 
                                                 intercept = True, 
                                                 iterations = i)
    
    model_SVM = SVMWithSGD.train(training, regType="l2", regParam=0.01, 
                                 intercept = True, iterations = i)
    print 'LogReg:\t{}'.format(model_LR)
    print 'SVM:\t{}'.format(model_SVM)

    # Evaluating the model on training data
    labelsAndPreds_LR = testing.\
        map(lambda p: (p.label, model_LR.predict(p.features)))
    labelsAndPreds_SVM = testing.\
        map(lambda p: (p.label, model_SVM.predict(p.features)))
    trainErr_LR = labelsAndPreds_LR.\
        filter(lambda (v, p): v != p).count() / float(testing.count())
    trainErr_SVM = labelsAndPreds_SVM.\
        filter(lambda (v, p): v != p).count() / float(testing.count())
    print "Testing Error:"
    print "\tLR:\t{}".format(str(trainErr_LR))
    print "\tSVM:\t{}".format(str(trainErr_SVM))
    print '-------------------'

The testing error is highly dependent on the value of $\lambda$, especially in the case of the SVM (which seems to converge also more slowly).

Though not plotted here, the final hyperplanes are quite similar: $w_0 \simeq  0, w_1 \simeq -w2$, so the lines are both diagonals almost crossing the origin.

## HW11.3.4

**Derive and implement in Spark a weighted  LASSO logistic regression. Implement a convergence test of your choice to check for termination within your training algorithm.**

The following function is general purpose: we just have to weight each point before passing it to the function as an argument. So I start analyzing the results of the unweighted version.

The convergence test is the same as before.

The function already checks the misclassification error, but for the moment we'll test the training set.

In [ ]:
def logRegressionGDReg(training, testing, wInitial=None, learningRate=0.05, 
                       iterations=50, regParam=0.01, regType=None):

    ## Draw the true hyperplane
    x1 = [-1, 1]
    w = [-1, 1, 0]
    x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
    plt.figure(figsize=(10,10))
    plt.plot(x1, x2, 'b', label="True line", linewidth=2.0, color = 'black', 
             linestyle='--')

    featureLen = len(training.take(1)[0].x)
    n = training.count()
    if wInitial is None:
        w = np.random.normal(size=featureLen)
        #w = [0]*featureLen
        #w[-1] = 1
        #w = np.array(w)
    else:
        w = wInitial
    prev_w = np.array([float('inf')]*featureLen)
    #prev_Err = float('inf')
    Err_vector = []
    convergence = False
    
    x1 = [-1,1]
    colors = plt.cm.Set1(np.linspace(0, 1, iterations/25+1))
    
    for it in range(iterations+1):
        wBroadcast = sc.broadcast(w)
        gradient = training.\
            map(lambda p: (1 / (1 + np.exp(-p.y * np.dot(wBroadcast.value, 
                                                         p.x)))-1) * p.y * 
                np.array(p.x)).\
            reduce(lambda a, b: a + b)
        errors = testing.\
            map(lambda p: p.y != (np.dot(wBroadcast.value, p.x)>0).
                astype(int)*2 - 1).\
            filter(lambda wx: wx == True).count()
        Err = float(errors) / n
        Err_vector.append(Err)
        
        if Err == 0.1 and convergence == False:
            print '+++++++++++++++++++++++++++++++++++++++'
            print 'Minimum error achieved at iteration {}'.format(it)
            print 'Ratio of weights at this iteration:'
            print '\t{}'.format(ratio_of_ratio)
            print '+++++++++++++++++++++++++++++++++++++++'
            convergence = True

        if regType == "Ridge":
            wReg = w * 1
            wReg[-1] = 0 
        elif regType == "Lasso":
            wReg = w * 1
            wReg[-1] = 0 
            wReg = (wReg>0).astype(int) * 2-1
        else:
            wReg = np.zeros(w.shape[0])
        gradient = gradient + regParam * wReg
        ## Gradient: GD of Squared Error+ GD of regularized term 
        w = w - learningRate * gradient / n
        if it % 25 == 0 and it != 0:
            print 'Iteration {}'.format(str(it))
            print '\tWeights:\t{}'.format(w)
            print '\tError rate:\t{}'.format(str(Err))
            print '----------------------------'
            x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
            ## Plot the hyperplanes
            plt.plot(x1, x2, color=colors[it/25], label="After {} iterations".
                 format(str(it)), linewidth=1.5)
        
        ## STOP IF CONVERGENCE
        comb = [c for c in itertools.combinations(range(len(w)),2)]
        ratio_of_ratio = np.array([(w[i]/prev_w[i])/(w[j]/prev_w[j]) 
                                   for (i,j) in comb])
        condition = (0.99<=ratio_of_ratio) & (ratio_of_ratio<=1.01)
        if condition.all(axis=0):
            if Err != 0.1:
                if convergence == False:
                    print '+++++++++++++++++++++++++++++++++++++++++++'
                    print 'Converge criterion met after {} iterations'\
                        '\nbut lowest error not achieved yet.'.format(str(it))
                    print '+++++++++++++++++++++++++++++++++++++++++++'
                    convergence = True
            else:
                print 'Converged after {} iterations.'.format(str(it))
                print '\tWeights:\t{}'.format(w)
                print '\tError rate:\t{}'.format(str(Err))
                x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
                plt.plot(x1, x2, color='red', label="After {} iterations".
                         format(str(it)), linewidth=2.0)
                plt.legend(bbox_to_anchor=(1.05, 1), loc=2, fontsize=20, 
                           borderaxespad=0.)
                plt.xlabel("x1")
                plt.ylabel("x2")
                axes = plt.gca()
                axes.set_xlim([-1,1])
                axes.set_ylim([-1,1])
                plt.grid()
                plt.show()
                break
        prev_w = w
        #prev_Err = Err
        if it == iterations:
            print 'Not enough iterations to converge'
            plt.legend(bbox_to_anchor=(1.05, 1), loc=2, fontsize=20, 
                       borderaxespad=0.)
            plt.xlabel("x1")
            plt.ylabel("x2")
            axes = plt.gca()
            axes.set_xlim([-1,1])
            axes.set_ylim([-1,1])
            plt.grid()
            plt.show()
    return Err_vector

In [ ]:
from collections import namedtuple
Point = namedtuple('Point', 'x y')

def readPoint(line):
    d = line.split(',')
    x = [float(i) for i in d[:2]]
    x.append(1.0)  #bias term
    return Point(x, float(d[2]))

train = sc.textFile('training_set.csv').map(readPoint).cache()

np.random.seed(12345)

Error_Vector_LR1 = logRegressionGDReg(train, train, iterations=200, 
                                      regParam=0.01, regType="Lasso")

In [ ]:
num_iter = len(Error_Vector_LR1)
plt.figure(figsize=(10,10))
plt.plot(range(num_iter), Error_Vector_LR1)
plt.xlabel("Iteration")
plt.ylabel("Error")
axes = plt.gca()
#axes.set_ylim([np.floor(min(Error_Vector_LR1)*20)/20-.05, 
#               np.ceil(max(Error_Vector_LR1)*20)/20+.05])
#axes.set_xlim(0, num_iter)
axes.set_ylim(0,0.55)
axes.set_xlim(0,200)
plt.grid()
plt.show()

The unweighted version of the homegrown code has the lowest misclasiffication error at the 140th iteration (compared to the 2nd in the MLLib version), though it reaches convergence (based on the criterion previously described) before that, at the 136th iteration (compared to the 10th in the MLLib version).

The hyperplane is not exaclty the true one (but that's highly dependent on the random datase we generated).

## HW11.3.5
**Weight the above training dataset as follows:  Weight each example using the inverse vector length (Euclidean norm):**
        
    weight(X)= 1/||X||

**where**

    ||X|| = SQRT(X.X)= SQRT(X1^2 + X2^2)

**Here X is vector made up of X1 and X2.**

To weight the training dataset we just have to apply the weights before calling the function defined before.

In [ ]:
def readPoint_W(line):
    d = line.split(',')
    x = [float(i) for i in d[:2]]
    x = list(x/np.sqrt(np.sum(np.array(x)**2)))
    x.append(1.0)  #bias term
    return Point(x, float(d[2]))

train = sc.textFile('training_set.csv').map(readPoint_W).cache()

np.random.seed(12345)

Error_Vector_LR2 = logRegressionGDReg(train, train, 
                                      iterations=200, regParam=0.01, 
                                      regType="Lasso")

In [ ]:
num_iter = len(Error_Vector_LR2)
plt.figure(figsize=(10,10))
plt.plot(range(num_iter), Error_Vector_LR2)
plt.xlabel("Iteration")
plt.ylabel("Error")
axes = plt.gca()
axes.set_ylim(0,0.55)
axes.set_xlim(0,200)
plt.grid()
plt.show()

The weighted version converges at the 171st iteration (vs. 52nd in the unweighted version). And the minimum misclassification error (in the training set!) is achieved with quite less iterations: 21 instead of 140.

## HW11.3.6
**Evaluate your homegrown weighted  LASSO logistic regression on the test dataset. Report misclassification error (1 - Accuracy) and how many iterations does it took to converge.**

Now we just have to use the testing set (instead of the training set) as the 2nd parameter.

In [ ]:
test = sc.textFile('testing_set.csv').map(readPoint_W).cache()

np.random.seed(12345)

Error_Vector_LR3 = logRegressionGDReg(train, test, 
                                      iterations=200, regParam=0.01, 
                                      regType="Lasso")

In [ ]:
num_iter = len(Error_Vector_LR3)
plt.figure(figsize=(10,10))
plt.plot(range(num_iter), Error_Vector_LR3)
plt.xlabel("Iteration")
plt.ylabel("Error")
axes = plt.gca()
axes.set_ylim(0,0.55)
axes.set_xlim(0,250)
plt.grid()
plt.show()

The convergence criterion is again met at the 171st iteration (the initial weights are random, but we've used the same seed), and the minimum misclassification error (even in the testing set) is reached at the 20th iteration (21st in the training set).

**Does Spark MLLib have a weighted LASSO logistic regression implementation. If so use it and report your findings on the weighted training set and test set.**

I haven't found it, but it's just a matter of adapting the driver code a bit.

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, \
    LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
import itertools

iterations = 20
## Define equally spaced colors for each iteration
colors = plt.cm.Set1(np.linspace(0, 1, iterations+1))

def parsePoint_W(line):
    ## Change labels to 0 and 1 (rather than -1 and 1)
        ## Those are the values accepted by MLLib
    values = [float(x) for x in line.split(',')]
    x = list(values[:2]/np.sqrt(np.sum(np.array(values[:2])**2)))
    if values[2] == -1:
        y = 0
    elif values[2] == 1:
        y = 1
    return LabeledPoint(y, x)

## Load and cache both sets
training = sc.textFile("training_set.csv")
testing = sc.textFile("testing_set.csv")
training = training.map(parsePoint_W).cache()
testing = testing.map(parsePoint_W).cache()

## Draw the true hyperplane
x1 = [-1, 1]
w = [-1, 1, 0]
x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
plt.figure(figsize=(10,10))
plt.plot(x1, x2, 'b', label="True line", linewidth=2.0, color = 'black', 
         linestyle='--')

prev_w = [float('inf')]*3
#prev_Err = float('inf')

## Try a few iterations (no need for more in this case)
for it in range(iterations+1):
    print 'Iteration: {}'.format(it)
    # Build the model (Lasso/L1, lambda = 0.5)
    model = LogisticRegressionWithLBFGS.train(training, regType="l1", 
                                              regParam=0.01, 
                                              intercept = True, 
                                              iterations = it)
    print model

    # Evaluating the model on testing data
    labelsAndPreds = testing.\
        map(lambda p: (p.label, model.predict(p.features)))
    Err = labelsAndPreds.\
        filter(lambda (v, p): v != p).count() / float(testing.count())
    w = [model.weights[0], model.weights[1], model.intercept]
    x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
    ## Plot the hyperplanes
    plt.plot(x1, x2, color=colors[it], label="After {} iterations".
             format(str(it)), linewidth=1.5)
    
    ## Stop when it converges
    comb = [c for c in itertools.combinations(range(len(w)),2)]
    ratio_of_ratio = np.array([(w[i]/prev_w[i])/(w[j]/prev_w[j]) 
                               for (i,j) in comb])
    condition = (0.99<=ratio_of_ratio) & (ratio_of_ratio<=1.01)
    if condition.all(axis=0):
        break
    
    prev_w = w
    #prev_Err = Err
    print "Testing Error: {}".format(str(Err))
    print '------------------'

## Last iteration's results
print "Testing Error: {}".format(str(Err))
print '------------------'
print 'CONVERGENCE AFTER {} ITERATIONS'.format(it)
print 'Relative change of weights (incl. intercept) between iterations '\
    '{} and {}:\n\t{}'.format(it-1, it, [a/b for (a,b) in zip(w,prev_w)])

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, fontsize=20, borderaxespad=0.)
plt.xlabel("x1")
plt.ylabel("x2")
axes = plt.gca()
axes.set_ylim([-1,1])
axes.set_xlim([-1,1])
plt.grid()
plt.show()

Now it takes a few more iterations to converge (14 vs. 9, still less than with the homegrown code), but the minimum misclassification error in the testing set is achieved even with just 1 iteration.

# HW11.4: SVMs

**Use the non-linearly separable training and testing datasets from HW11.3 in this problem.**

## HW11.4.1
**Using MLLib  train up a soft SVM model with the training dataset and evaluate with the testing set. What is a good number of iterations for training the SVM model? Justify with plots and words.**

The criterion for convergence is the same as before. All the code is pretty much the same, we just have to use `SVMWithSGD` instead of `LogisticRegressionWithLBFGS`.

This time I've chosen L2 regularization, the default for linear SVMs.

In [ ]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

iterations = 40
## Define equally spaced colors for each iteration
colors = plt.cm.Set1(np.linspace(0, 1, iterations/2+1))

def parsePoint(line):
    ## Change labels to 0 and 1 (rather than -1 and 1)
        ## Those are the values accepted by MLLib
    values = [float(x) for x in line.split(',')]
    if values[2] == -1:
        y = 0
    elif values[2] == 1:
        y = 1
    return LabeledPoint(y, values[:2])

## Load and cache both sets
training = sc.textFile("training_set.csv")
testing = sc.textFile("testing_set.csv")
training = training.map(parsePoint).cache()
testing = testing.map(parsePoint).cache()

## Draw the true hyperplane
x1 = [-1, 1]
w = [-1, 1, 0]
x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
plt.figure(figsize=(10,10))
plt.plot(x1, x2, 'b', label="True line", linewidth=2.0, color = 'black', 
         linestyle='--')

prev_w = [float('inf')]*3
#prev_Err = float('inf')

## Try a few iterations (no need for more in this case)
for it in range(iterations+1):
    print 'Iteration: {}'.format(it)
    # Build the model (Lasso/L1, lambda = 0.5)
    model = SVMWithSGD.train(training, regType="l2", regParam=0.01, 
                             intercept = True, iterations = it)
    print model

    # Evaluating the model on testing data
    labelsAndPreds = testing.\
        map(lambda p: (p.label, model.predict(p.features)))
    Err = labelsAndPreds.\
        filter(lambda (v, p): v != p).count() / float(testing.count())
    w = [model.weights[0], model.weights[1], model.intercept]
    if it % 2 == 0 and it != 0:
        x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
        ## Plot the hyperplanes
        plt.plot(x1, x2, color=colors[it/2], label="After {} iterations".
                 format(str(it)), linewidth=1.5)
    
    comb = [c for c in itertools.combinations(range(len(w)),2)]
    ratio_of_ratio = np.array([(w[i]/prev_w[i])/(w[j]/prev_w[j]) 
                               for (i,j) in comb])
    condition = (0.99<=ratio_of_ratio) & (ratio_of_ratio<=1.01)
    if condition.all(axis=0):
        break
    
    prev_w = w
    #prev_Err = Err
    print "Testing Error: {}".format(str(Err))
    print '------------------'

## Last iteration's results
print "Testing Error: {}".format(str(Err))
print '------------------'
print 'CONVERGENCE AFTER {} ITERATIONS'.format(it)
print 'Relative change of weights (incl. intercept) between iterations '\
    '{} and {}:\n\t{}'.format(it-1, it, [a/b for (a,b) in zip(w,prev_w)])

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, fontsize=20, borderaxespad=0.)
plt.xlabel("x1")
plt.ylabel("x2")
axes = plt.gca()
axes.set_ylim([-1,1])
axes.set_xlim([-1,1])
plt.grid()
plt.show()

It takes more iterations to converge than with the MLLib version of Logistic Regression: the minimum error is achieved at the 6th iteration, and the converge criterion is met at the 35th iteration.

## HW11.4.2
**Derive and Implement in Spark a weighted soft linear svm classification learning algorithm.**

Again, I define a general-purpose function that works with weighted or unweighted data, uses the convergence criterion describe previously, and so on. The main changes apply of course to the loss function, the gradient, and the predictions.

In [ ]:
def SVM_GDReg(training, testing, wInitial=None, 
                         learningRate=0.05, iterations=50, regParam=0.1):
        
    ## Draw the true hyperplane
    x1 = [-1, 1]
    w = [-1, 1, 0]
    x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
    plt.figure(figsize=(10,10))
    plt.plot(x1, x2, 'b', label="True line", linewidth=2.0, color = 'black', 
             linestyle='--')

    featureLen = len(training.take(1)[0].x)
    n = training.count()
    if wInitial is None:
        w = np.random.normal(size=featureLen)
    else:
        w = wInitial
    prev_w = np.array([float('inf')]*featureLen)
    #prev_Err = float('inf')
    Err_vector = []
    Num_SVs = []
    convergence = False
    
    x1 = [-1,1]
    colors = plt.cm.Set1(np.linspace(0, 1, iterations/10+1))
    
    for it in range(iterations+1):
        wBroadcast = sc.broadcast(w)
        sv = training.\
            filter(lambda p: p.y * np.dot(wBroadcast.value, p.x) < 1)
            ## Only support vectors (label*margin) < 1
        if sv.isEmpty(): ## Converged as no more updates possible
            print 'No more support vectors at iteration {}'.\
                format(str(it))
            plt.legend(bbox_to_anchor=(1.05, 1), loc=2, fontsize=20, 
                       borderaxespad=0.)
            plt.xlabel("x1")
            plt.ylabel("x2")
            axes = plt.gca()
            axes.set_xlim([-1,1])
            axes.set_ylim([-1,1])
            plt.grid()
            plt.show()
            break ## Hinge loss component of gradient y*x and sum up 
        gradient = sv.map(lambda p: -p.y * np.array(p.x)).\
            reduce(lambda a,b: a + b) / n ## Gradient = average hinge loss
        num_sv = sv.count()
        
        ## PREDICTIONS: sign(w^T*x)
        errors = testing.\
            map(lambda p: p.y != np.sign(np.dot(wBroadcast.value, p.x))).\
            filter(lambda pred: pred == True).count()
        Err = float(errors) / n
        Err_vector.append(Err)
        Num_SVs.append(num_sv)
        
        if Err == 0.1 and convergence == False:
            print '+++++++++++++++++++++++++++++++++++++++'
            print 'Minimum error achieved at iteration {}'.format(it)
            print 'Ratio of weights at this iteration:'
            print '\t{}'.format(ratio_of_ratio)
            print '+++++++++++++++++++++++++++++++++++++++'
            convergence = True

        wReg = w*1 ## wReg = w in Ridge/L2
        wReg[-1] = 0
        ## Gradient: hinge loss + regularized term 
        wDelta = learningRate*(gradient + regParam*wReg)

        if it % 25 == 0 and it != 0:
            print 'Iteration {}'.format(str(it))
            print '\tSupport vectors:\t{}'.format(str(num_sv))
            print '\tWeights:\t\t{}'.format(w)
            print '\tError rate:\t\t{}'.format(str(Err))
            print '----------------------------'
            x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
            ## Plot the hyperplanes
            plt.plot(x1, x2, color=colors[it/25], label="After {} iterations".
                 format(str(it)), linewidth=1.5)        
        
        comb = [c for c in itertools.combinations(range(len(w)),2)]
        ratio_of_ratio = np.array([(w[i]/prev_w[i])/(w[j]/prev_w[j]) 
                                   for (i,j) in comb])
        condition = (0.99<=ratio_of_ratio) & (ratio_of_ratio<=1.01)
        if condition.all(axis=0):
            if Err != 0.1 and convergence == False:
                print '+++++++++++++++++++++++++++++++++++++++++++'
                print 'Converge criterion met after {} iterations'\
                    'but lowest error not achieved yet.'.format(str(it))
                print '+++++++++++++++++++++++++++++++++++++++++++'
                convergence = True
            else:
                print 'Converged after {} iterations.'.format(str(it))
                print '\tSupport vectors:\t{}'.format(str(num_sv))
                print '\tWeights:\t\t{}'.format(w)
                print '\tError rate:\t\t{}'.format(str(Err))
                x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
                plt.plot(x1, x2, color='red', label="After {} iterations".
                         format(str(it)), linewidth=2.0)
                plt.legend(bbox_to_anchor=(1.05, 1), loc=2, fontsize=20, 
                           borderaxespad=0.)
                plt.xlabel("x1")
                plt.ylabel("x2")
                axes = plt.gca()
                axes.set_xlim([-1,1])
                axes.set_ylim([-1,1])
                plt.grid()
                plt.show()
                break
        prev_w = w
        w = w - wDelta
        
    return (Num_SVs, Err_vector)

Let's see the results using the unweighted data and checking the error in the training set.

In [ ]:
train = sc.textFile('training_set.csv').map(readPoint).cache()

np.random.seed(12345)
Error_Vector_SVM1 = SVM_GDReg(train, train, learningRate = 0.05, 
                              iterations=100, regParam=0.01)

In [ ]:
num_iter = len(Error_Vector_SVM1[0])

plt.figure(figsize=(10,10))
plt.plot(range(num_iter), Error_Vector_SVM1[1])
plt.xlabel("Iteration")
plt.ylabel("Error")
axes = plt.gca()
axes.set_ylim(0,0.55)
axes.set_xlim(0,200)
plt.grid()
plt.show()

plt.figure(figsize=(10,10))
plt.plot(range(num_iter), Error_Vector_SVM1[0])
plt.xlabel("Iteration")
plt.ylabel("Number of Support Vectors")
axes = plt.gca()
axes.set_ylim(0, 100)
axes.set_xlim(0, 200)
plt.grid()
plt.show()

Unlike the MLLib version, which took more iterations to converge (compared to Logistic Regression), the homegrown code takes less: 49 to get the minimum misclassification error, and 66 to meet the convergence criterion that I defined. We end up with 50 support vectors (of the possible 100, though the 10 mislabeled points should not be support vectors at convergence).

Let's see what happens when weights are applied (again, using the `readPoint_W` function and applying the weighted data to the ML function):

In [ ]:
train = sc.textFile('training_set.csv').map(readPoint_W).cache()

np.random.seed(12345)
Error_Vector_SVM2 = SVM_GDReg(train, train, learningRate = 0.05, 
                              iterations=100, regParam=0.01)

In [ ]:
num_iter = len(Error_Vector_SVM2[0])

plt.figure(figsize=(10,10))
plt.plot(range(num_iter), Error_Vector_SVM2[1])
plt.xlabel("Iteration")
plt.ylabel("Error")
axes = plt.gca()
axes.set_ylim(0,0.55)
axes.set_xlim(0,200)

plt.grid()
plt.show()

plt.figure(figsize=(10,10))
plt.plot(range(num_iter), Error_Vector_SVM2[0])
plt.xlabel("Iteration")
plt.ylabel("Number of Support Vectors")
axes = plt.gca()
axes.set_ylim(0, 100)
axes.set_xlim(0, 200)
plt.grid()
plt.show()

Now the SVM converges even faster: after 9 iterations we have an error (in the training set) of 10%, and with 53 we've met the convergence criterion.

The number of support vectors with weighted data is reduced to 25.

## HW11.4.3
**Evaluate your homegrown weighted soft linear svm classification learning algorithm on the weighted training dataset and test dataset from HW11.3. Report misclassification error (1 - Accuracy) and how many iterations does it took to converge?  How many support vectors do you end up?**

In [ ]:
test = sc.textFile('testing_set.csv').map(readPoint_W).cache()

np.random.seed(12345)
Error_Vector_SVM3 = SVM_GDReg(train, test, learningRate = 0.05, 
                              iterations=100, regParam=0.01)

In [ ]:
num_iter = len(Error_Vector_SVM3[0])

plt.figure(figsize=(10,10))
plt.plot(range(num_iter), Error_Vector_SVM3[1])
plt.xlabel("Iteration")
plt.ylabel("Error")
axes = plt.gca()
axes.set_ylim(0,0.55)
axes.set_xlim(0,200)
plt.grid()
plt.show()

plt.figure(figsize=(10,10))
plt.plot(range(num_iter), Error_Vector_SVM3[0])
plt.xlabel("Iteration")
plt.ylabel("Number of Support Vectors")
axes = plt.gca()
axes.set_ylim(0, 100)
axes.set_xlim(0, 200)
plt.grid()
plt.show()

Even with the test set after 9 iterations (again) we get the lowest error possible: 10%. As mentioned in HW11.3, the iterations to meet the convergence criterion are the same than before because I used the same seed to generate the initial weights.

Again (this depends on the convergence criterion, not on the training or testing error) we end up with only 25 support vectors.

## HW11.4.4
**Does Spark MLLib have a weighted soft SVM learner. If so use it and report your findings on the weighted training set and test set.**

Again, it doesn't seem so (http://apache-spark-user-list.1001560.n3.nabble.com/MLlib-weighted-training-td10291.html) but we can adapt the previous code.

In [ ]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
import itertools

iterations = 15
## Define equally spaced colors for each iteration
colors = plt.cm.Set1(np.linspace(0, 1, iterations+1))

def parsePoint_W(line):
    ## Change labels to 0 and 1 (rather than -1 and 1)
        ## Those are the values accepted by MLLib
    values = [float(x) for x in line.split(',')]
    x = list(values[:2]/np.sqrt(np.sum(np.array(values[:2])**2)))
    if values[2] == -1:
        y = 0
    elif values[2] == 1:
        y = 1
    return LabeledPoint(y, x)

## Load and cache both sets
training = sc.textFile("training_set.csv")
testing = sc.textFile("testing_set.csv")
training = training.map(parsePoint_W).cache()
testing = testing.map(parsePoint_W).cache()

## Draw the true hyperplane
x1 = [-1, 1]
w = [-1, 1, 0]
x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
plt.figure(figsize=(10,10))
plt.plot(x1, x2, 'b', label="True line", linewidth=2.0, color = 'black', 
         linestyle='--')

prev_w = [float('inf')]*3
#prev_Err = float('inf')

## Try a few iterations (no need for more in this case)
for it in range(iterations+1):
    print 'Iteration: {}'.format(it)
    # Build the model (Lasso/L1, lambda = 0.5)
    model = SVMWithSGD.train(training, regType="l2", regParam=0.01, 
                             intercept = True, iterations = it)
    print model

    # Evaluating the model on testing data
    labelsAndPreds = testing.\
        map(lambda p: (p.label, model.predict(p.features)))
    Err = labelsAndPreds.\
        filter(lambda (v, p): v != p).count() / float(testing.count())
    w = [model.weights[0], model.weights[1], model.intercept]
    x2 = [-(i * w[0] + w[2]) / w[1] for i in x1]
    ## Plot the hyperplanes
    plt.plot(x1, x2, color=colors[it], label="After {} iterations".
             format(str(it)), linewidth=1.5)
    
    comb = [c for c in itertools.combinations(range(len(w)),2)]
    ratio_of_ratio = np.array([(w[i]/prev_w[i])/(w[j]/prev_w[j]) 
                               for (i,j) in comb])
    condition = (0.99<=ratio_of_ratio) & (ratio_of_ratio<=1.01)
    if condition.all(axis=0):
        break
    
    prev_w = w
    #prev_Err = Err
    print "Testing Error: {}".format(str(Err))
    print '------------------'

## Last iteration's results
print "Testing Error: {}".format(str(Err))
print '------------------'
print 'CONVERGENCE AFTER {} ITERATIONS'.format(it)
print 'Relative change of weights (incl. intercept) between iterations '\
    '{} and {}:\n\t{}'.format(it-1, it, [a/b for (a,b) in zip(w,prev_w)])

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, fontsize=20, borderaxespad=0.)
plt.xlabel("x1")
plt.ylabel("x2")
axes = plt.gca()
axes.set_ylim([-1,1])
axes.set_xlim([-1,1])
plt.grid()
plt.show()

# SUMMARY

|LR|MLLib|homegrown|hw weighted|hw weighted w/ test|MLLib weighted|
|:---:|:---:|:---:|:---:|:---:|:---:|
|Iterations until Min. Error|2|140|21|20|1|
|Iterations until Conv. crit.|10|136|171|171|14|

|SVM|MLLib|homegrown|hw weighted|hw weighted w/ test|MLLib weighted|
|:---:|:---:|:---:|:---:|:---:|:---:|
|Iterations until Min. Error|6|49|9|9|2|
|Iterations until Conv. crit.|35|66|53|53|10|
|Support Vectors|-|50|25|25|-|